In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [36]:
url = 'https://www.ccny.cuny.edu/registrar/fall'
response = requests.get(url)

In [3]:
response

<Response [200]>

In [37]:
soup = BeautifulSoup(response.text, 'html.parser')

print(soup.prettify())

calendar_table = soup.find('tbody')

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <meta content="40.8200471,-73.9492724" name="icbm"/>
  <meta content="US-NY" name="geo.region"/>
  <meta content="The City College of New York" name="geo.placename"/>
  <meta content="40.8200471,-73.9492724" name="geo.position"/>
  <link href="https://www.ccny.cuny.edu/registrar/fall" rel="canonical"/>
  <meta content="index, follow" name="robots"/>
  <meta content="en-us" http-equiv="content-language"/>
  <meta content="Drupal 9" name="generator"/>
  <meta content="The City College of New York" property="og:site_name"/>
  <meta content="text

In [38]:
def date_handler(s):
    s = s.strip()

    if not s or s == "2021":
        return None

    if " - " in s:
        start_part, end_day = s.split(" - ")
        start_date_obj = datetime.strptime(f"{start_part} 2021", "%B %d %Y")
        return f"{start_date_obj.year}-{start_date_obj.month:02d}-({start_date_obj.day} - {end_day})"
    else:
        if "2022" in s or "," in s:
            single_date_obj = datetime.strptime(s, "%B %d, %Y")
        else:
            single_date_obj = datetime.strptime(f"{s} 2021", "%B %d %Y")
        return single_date_obj.strftime("%Y-%m-%d")

In [39]:
calendar = []

In [40]:
for tr in calendar_table.find_all('tr'):
    tds = tr.find_all('td')

    if len(tds) == 3:
        date_col = tds[0].get_text(strip=True)
        dow_col = tds[1].get_text(strip=True)
        event_col = tds[2].get_text(strip=True)

        formatted_date = date_handler(date_col)

        if formatted_date:
            calendar.append({
                "date": formatted_date,
                "dow": dow_col,
                "text": event_col
            })

In [42]:
df = pd.DataFrame(calendar)

df.set_index('date', inplace=True)

In [43]:
display(df)

,dow,text
date,,
2021-08-01,Sunday,Application for degree for January and Februar...
2021-08-18,Wednesday,Last day to apply for Study Abroad
2021-08-24,Tuesday,Last day of Registration;Last day to file ePer...
2021-08-25,Wednesday,Start of Fall Term;Classes begin;Initial Regis...
2021-08-(25 - 31),Wednesday - Tuesday,Change of program period; late fees apply
2021-08-26,Thursday,Last day for Independent Study
2021-08-28,Saturday,First day of Saturday Classes
2021-08-31,Tuesday,Last day to add a class to an existing enrollm...
2021-09-01,Wednesday,Verification of Enrollment rosters available t...
